In [5]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from heat_1d_2d import heat_1d_nn, heat_2d_nn

np.random.seed(238)
torch.manual_seed(301)

In [ ]:
layers = [32, 64, 128, 128, 1]
#in this example: the output layer uses identity as activation func and all the hidden layers use tanh
activations = [torch.tanh]*(len(layers)-1) + [None]
pde_nn = heat_1d_nn(layers, activations)

N_colloc = 100

x = np.linspace(0, 1, N_colloc)
y = np.linspace(0, 1, N_colloc)

xs, ys = np.meshgrid(x, y)

xs = torch.tensor(xs, dtype=torch.float32).view(-1, 1)
ys = torch.tensor(ys, dtype=torch.float32).view(-1, 1)

x_colloc = torch.tensor(xs, dtype=torch.float32).view(-1,1)
t_colloc = torch.tensor(ys, dtype=torch.float32).view(-1,1)

# gives relatively good results (compared to other parameters, still bad though)
pde_nn.train(x_colloc, t_colloc, lr=1e-2, weight_decay=0.0, epochs = 200, print_epochs=50)
#proper result for epochs =5000
pde_nn.train_lbfgs(x_colloc, t_colloc, lr=1, epochs=50, max_iter=50)

for t in [0.001, 0.005, 0.01, 0.1, 0.2, 0.3, 0.5]:
    x_test = torch.linspace(0,1,100).view(-1,1)
    t_test = torch.tensor([[t]]*100)  # t=0.5
    u_pred = pde_nn.trial_solution(x_test, t_test).detach().numpy()


    #compare results with analytic solution

    x_np = x_test.numpy().flatten()  # convert to 1D array for plotting
    t_val = t
    u_analytic = np.sin(np.pi * x_np) * np.exp(-np.pi**2 * t_val) + np.sin(4 * np.pi * x_np
                    ) * np.exp(- 16 * np.pi**2 * t_val) 

    # Plot the results
    plt.figure(figsize=(8,5))
    plt.plot(x_np, u_pred, label='NN Prediction', linewidth=2)
    plt.plot(x_np, u_analytic, '--', label='Analytic Solution', linewidth=2)
    plt.xlabel('x')
    plt.ylabel(f'u(x,t={t})')
    plt.ylim(-0.75, 2.0)
    plt.title('Heat Equation: NN vs Analytic Solution')
    plt.legend()
    plt.grid(True)
    plt.show()

/tmp/ipykernel_671/3175112879.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_colloc = torch.tensor(xs, dtype=torch.float32).view(-1,1)
/tmp/ipykernel_671/3175112879.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t_colloc = torch.tensor(ys, dtype=torch.float32).view(-1,1)


Epoch 0, Loss: 5301.728027
Epoch 50, Loss: 3384.761230
Epoch 100, Loss: 2993.624756
Epoch 150, Loss: 2332.745361
Epoch 200, Loss: 1261.173950
Epoch 250, Loss: 877.925598
Epoch 300, Loss: 683.239380
Epoch 350, Loss: 536.054932
Epoch 400, Loss: 462.274261
Epoch 450, Loss: 412.965546
Epoch 500, Loss: 374.912964
Epoch 550, Loss: 355.845490
Epoch 600, Loss: 338.450928
Epoch 650, Loss: 310.489105
Epoch 700, Loss: 300.727356
Epoch 750, Loss: 296.453674
Epoch 800, Loss: 270.538086
Epoch 850, Loss: 278.437286
Epoch 900, Loss: 248.521927
Epoch 950, Loss: 242.132568
Epoch 1000, Loss: 255.382202
Epoch 1050, Loss: 227.875153
Epoch 1100, Loss: 245.669800
Epoch 1150, Loss: 211.425751
Epoch 1200, Loss: 207.075134
Epoch 1250, Loss: 202.485382
Epoch 1300, Loss: 219.344971
Epoch 1350, Loss: 198.770203
Epoch 1400, Loss: 183.908768
Epoch 1450, Loss: 222.000168
Epoch 1499, Loss: 179.621948


NameError: name 'loss' is not defined

In [ ]:
"""
2D Heat equation
"""

layers = [200, 100, 50, 1]
activations = [torch.tanh]*(len(layers)-1) + [None]


N_colloc = 10000
x_colloc = torch.rand(N_colloc, 1)
y_colloc = torch.rand(N_colloc, 1)
t_colloc = torch.rand(N_colloc, 1)

pde_nn = heat_2d_nn(layers, activations)

pde_nn.train(x_colloc, y_colloc, t_colloc, lr=1e-3, weight_decay=0, epochs = 1_000, print_epochs=100)
#proper result for epochs =5000

x_test = torch.linspace(0,1,100).view(-1,1)
y_test = torch.linspace(0,1,100).view(-1,1)
t_test = torch.tensor([[0.001]]*100)  # t=0.001
u_pred = pde_nn.trial_solution(x_test, y_test, t_test).detach().numpy()


#compare results with analytic solution

x_np = x_test.numpy().flatten()  # convert to 1D array for plotting
y_np = y_test.numpy().flatten()
t_val = 0.5
u_analytic = np.sin(np.pi * x_np) * np.sin(np.pi * y_np) * np.exp(
    -2 * np.pi**2 * t_val) + np.sin(2 * np.pi * x_np) * np.sin(4 * np.pi * x_np
    ) * np.exp(-20 * np.pi**2 * t_val)

Epoch 0, Loss: 9729.363281
Epoch 100, Loss: 9321.371094
Epoch 200, Loss: 9107.053711
Epoch 300, Loss: 8792.874023
Epoch 400, Loss: 7021.835938
Epoch 500, Loss: 4788.136230
Epoch 600, Loss: 4008.713623
Epoch 700, Loss: 3217.027588
Epoch 800, Loss: 2766.715088
Epoch 900, Loss: 2488.868896
Epoch 999, Loss: 2270.819092
